## Pizza Sales Dashboard Validation

In [24]:
import pandas as pd
import numpy as np
import os
import datetime as dt

In [2]:
# create path to csvs
path = 'pizza_store_tables'


In [3]:
csv_list = os.listdir(path)
csv_list

['.ipynb_checkpoints',
 'orders.csv',
 'order_details - Copy.csv',
 'order_details.csv',
 'pizzas.csv',
 'pizza_types - Copy.csv',
 'pizza_types.csv']

### Read in the tables

In [4]:
# read in orders table
order_path = os.path.join(path,'orders.csv')
order_path
orders_df = pd.read_csv(order_path)
orders_df.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [5]:
# read in order_details table
order_details_path = os.path.join(path,'order_details.csv')
order_details_df = pd.read_csv(order_details_path)
order_details_df.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


In [6]:
# read in pizzas csv
pizzas_path = os.path.join(path,'pizzas.csv')
pizzas_df = pd.read_csv(pizzas_path)
pizzas_df.head()

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75


In [7]:
# read in pizza_types csv
pizza_types_df = pd.read_csv(os.path.join(path,'pizza_types.csv'),encoding= 'unicode_escape')
pizza_types_df.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


### Analysis and Dashboard Data Validation

In [8]:
# combine date and time object fields and convert to date time
orders_df['order_date'] = orders_df['date'] + ' ' + orders_df['time']
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
orders_df.head()

,order_id,date,time,order_date
0,1,2015-01-01,11:38:36,2015-01-01 11:38:36
1,2,2015-01-01,11:57:40,2015-01-01 11:57:40
2,3,2015-01-01,12:12:28,2015-01-01 12:12:28
3,4,2015-01-01,12:16:31,2015-01-01 12:16:31
4,5,2015-01-01,12:21:30,2015-01-01 12:21:30


#### Validate Date Range of Dashboard (Jan-Dec 2015)

In [9]:
print(orders_df['date'].min(),orders_df['date'].max())

2015-01-01 2015-12-31


#### Validate the times of day of ordering from heat map on dashboard

In [10]:
print(orders_df['time'].min(),orders_df['time'].max())

09:52:21 23:05:52


### Calculate Sales Dollars, No. of Orders, and No of Pizzas sold for each month to check dynamic scorecards on dashboard

In [11]:
# merge orders_df and order_details_df
orders_df2 = pd.merge(left=orders_df,right=order_details_df,how='inner',on='order_id')
orders_df2.head()

,order_id,date,time,order_date,order_details_id,pizza_id,quantity
0,1,2015-01-01,11:38:36,2015-01-01 11:38:36,1,hawaiian_m,1
1,2,2015-01-01,11:57:40,2015-01-01 11:57:40,2,classic_dlx_m,1
2,2,2015-01-01,11:57:40,2015-01-01 11:57:40,3,five_cheese_l,1
3,2,2015-01-01,11:57:40,2015-01-01 11:57:40,4,ital_supr_l,1
4,2,2015-01-01,11:57:40,2015-01-01 11:57:40,5,mexicana_m,1


In [12]:
# merge orders_df2 and pizzas_df to bring in prices for each line item
line_item_df = pd.merge(left=orders_df2,right=pizzas_df,how='inner',on='pizza_id')
line_item_df.head()

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price
0,1,2015-01-01,11:38:36,2015-01-01 11:38:36,1,hawaiian_m,1,hawaiian,M,13.25
1,77,2015-01-02,12:22:46,2015-01-02 12:22:46,179,hawaiian_m,1,hawaiian,M,13.25
2,146,2015-01-03,14:22:10,2015-01-03 14:22:10,357,hawaiian_m,1,hawaiian,M,13.25
3,163,2015-01-03,16:54:54,2015-01-03 16:54:54,389,hawaiian_m,1,hawaiian,M,13.25
4,247,2015-01-04,20:55:29,2015-01-04 20:55:29,568,hawaiian_m,1,hawaiian,M,13.25


In [13]:
# create new column to show total cost for each line item
line_item_df['line_cost']=line_item_df['quantity'] * line_item_df['price']
line_item_df.head()

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price,line_cost
0,1,2015-01-01,11:38:36,2015-01-01 11:38:36,1,hawaiian_m,1,hawaiian,M,13.25,13.25
1,77,2015-01-02,12:22:46,2015-01-02 12:22:46,179,hawaiian_m,1,hawaiian,M,13.25,13.25
2,146,2015-01-03,14:22:10,2015-01-03 14:22:10,357,hawaiian_m,1,hawaiian,M,13.25,13.25
3,163,2015-01-03,16:54:54,2015-01-03 16:54:54,389,hawaiian_m,1,hawaiian,M,13.25,13.25
4,247,2015-01-04,20:55:29,2015-01-04 20:55:29,568,hawaiian_m,1,hawaiian,M,13.25,13.25


In [14]:
# group by order_id to get quantity and total sales dollars per order
order_df3 = line_item_df.groupby(['order_id']).agg({'order_date':'max','quantity':'sum','line_cost':'sum'}).reset_index()
order_df3.head()

,order_id,order_date,quantity,line_cost
0,1,2015-01-01 11:38:36,1,13.25
1,2,2015-01-01 11:57:40,5,92.00
2,3,2015-01-01 12:12:28,2,37.25
3,4,2015-01-01 12:16:31,1,16.50
4,5,2015-01-01 12:21:30,1,16.50


In [15]:
monthly_sales_df = order_df3.resample('M',on='order_date').agg({'order_id':'count','quantity':'sum','line_cost':'sum'})
monthly_sales_df

,order_id,quantity,line_cost
order_date,,,
2015-01-31,1845,4232,69793.30
2015-02-28,1685,3961,65159.60
2015-03-31,1840,4261,70397.10
2015-04-30,1799,4151,68736.80
2015-05-31,1853,4328,71402.75
2015-06-30,1773,4107,68230.20
2015-07-31,1935,4392,72557.90
2015-08-31,1841,4168,68278.25
2015-09-30,1661,3890,64180.05


##### The No of Orders, No of Pizzas, and Total Sales Dollars per month match the results of the dynamic scorecard on the dashboard

## Sales by Size data validation

In [16]:
# group line_item_df by month, size to validate size sales chart
line_item_df.index=line_item_df['order_date']
line_item_df

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price,line_cost
order_date,,,,,,,,,,,
2015-01-01 11:38:36,1,2015-01-01,11:38:36,2015-01-01 11:38:36,1,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-01-02 12:22:46,77,2015-01-02,12:22:46,2015-01-02 12:22:46,179,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-01-03 14:22:10,146,2015-01-03,14:22:10,2015-01-03 14:22:10,357,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-01-03 16:54:54,163,2015-01-03,16:54:54,2015-01-03 16:54:54,389,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-01-04 20:55:29,247,2015-01-04,20:55:29,2015-01-04 20:55:29,568,hawaiian_m,1,hawaiian,M,13.25,13.25
...,...,...,...,...,...,...,...,...,...,...,...
2015-10-24 18:58:20,17492,2015-10-24,18:58:20,2015-10-24 18:58:20,39733,the_greek_xxl,1,the_greek,XXL,35.95,35.95
2015-11-05 20:28:52,18139,2015-11-05,20:28:52,2015-11-05 20:28:52,41183,the_greek_xxl,1,the_greek,XXL,35.95,35.95
2015-11-17 11:23:51,18789,2015-11-17,11:23:51,2015-11-17 11:23:51,42718,the_greek_xxl,1,the_greek,XXL,35.95,35.95


In [17]:
size_sales_df = line_item_df.groupby(by=[line_item_df.index.month,'size']).agg({'quantity':'sum','line_cost':'sum'})
size_sales_df

quantity  line_cost
order_date size                     
1          L         1640   32399.40
           M         1311   20943.50
           S         1229   15103.50
           XL          50    1275.00
           XXL          2      71.90
2          L         1521   30077.90
           M         1210   19263.00
           S         1189   14741.85
           XL          38     969.00
           XXL          3     107.85
3          L         1651   32752.35
           M         1313   20952.50
           S         1253   15538.90
           XL          41    1045.50
           XXL          3     107.85
4          L         1541   30622.70
           M         1390   22176.75
           S         1151   14136.05
           XL          65    1657.50
           XXL          4     143.80
5          L         1668   32970.50
           M         1338   21302.50
           S         1265   15644.90
           XL          54    1377.00
           XXL          3     107.85
6          L         1590   31493.80
           M         1335   21367.00
           S         1131   14048.00
           XL          49    1249.50
           XXL          2      71.90
7          L         1697   33583.05
           M         1396   22224.50
           S         1249   15475.35
           XL          50    1275.00
8          L         1527   30267.90
           M         1331   21175.50
           S         1264   15640.95
           XL          44    1122.00
           XXL          2      71.90
9          L         1511   29874.10
           M         1188   18956.75
           S         1152   14323.35
           XL          36     918.00
           XXL          3     107.85
10         L         1485   29338.60
           M         1253   19992.00
           S         1101   13554.10
           XL          42    1071.00
           XXL          2      71.90
11         L         1623   32226.20
           M         1337   21377.75
           S         1258   15536.05
           XL          45    1147.50
           XXL          3     107.85
12         L         1502   29712.20
           M         1233   19650.50
           S         1161   14333.50
           XL          38     969.00
           XXL          1      35.95

## Dashboard Heat Map Validation
- Initially after builing heat map viz, colors for sales dollars and tooltips did not match
- Colors updated when changing month parameter, but tooltips did not
- Sales color did not update when selecting different months

In [18]:
# create filtered table of only orders in December
december_line_item_df = line_item_df[line_item_df['date']>'2015-11-30']
december_line_item_df.head()

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price,line_cost
order_date,,,,,,,,,,,
2015-12-01 12:24:32,19680,2015-12-01,12:24:32,2015-12-01 12:24:32,44786,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-12-01 13:24:16,19690,2015-12-01,13:24:16,2015-12-01 13:24:16,44803,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-12-01 16:22:17,19704,2015-12-01,16:22:17,2015-12-01 16:22:17,44838,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-12-01 17:45:53,19712,2015-12-01,17:45:53,2015-12-01 17:45:53,44853,hawaiian_m,1,hawaiian,M,13.25,13.25
2015-12-01 19:21:05,19722,2015-12-01,19:21:05,2015-12-01 19:21:05,44875,hawaiian_m,1,hawaiian,M,13.25,13.25


In [19]:
# validate pizzas sold total in the new filtered table against above breakdown by month
december_line_item_df['quantity'].sum()

3935

In [20]:
# validate sales dollars of filtered table
december_line_item_df['line_cost'].sum()

64701.149999999994

In [21]:
december_line_item_df.describe()

,order_id,order_details_id,quantity,price,line_cost
count,3859.000000,3859.000000,3859.000000,3859.000000,3859.000000
mean,20511.754859,46691.000000,1.019694,16.442083,16.766300
std,490.732223,1114.141673,0.140818,3.579850,4.360662
min,19671.000000,44762.000000,1.000000,9.750000,9.750000
25%,20073.500000,45726.500000,1.000000,12.500000,12.750000
50%,20505.000000,46691.000000,1.000000,16.500000,16.500000
75%,20943.500000,47655.500000,1.000000,20.250000,20.500000
max,21350.000000,48620.000000,3.000000,35.950000,51.000000


In [22]:
# Check min and max order times of filtered table
print(december_line_item_df.time.max(),december_line_item_df.time.min())

23:03:23 11:02:20


##### Min and Max order times seem to align with the information from the tooltips for december

In [23]:
# filter full line_item_df for orders before 11:00:00 to confirm those orders exist but are all before December
line_item_df[line_item_df['time']<'11:00:00']

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price,line_cost
order_date,,,,,,,,,,,
2015-07-02 10:34:34,10862,2015-07-02,10:34:34,2015-07-02 10:34:34,24700,classic_dlx_m,1,classic_dlx,M,16.00,16.00
2015-05-07 10:54:15,7521,2015-05-07,10:54:15,2015-05-07 10:54:15,17097,thai_ckn_l,1,thai_ckn,L,20.75,20.75
2015-10-04 10:54:55,16439,2015-10-04,10:54:55,2015-10-04 10:54:55,37234,prsc_argla_l,1,prsc_argla,L,20.75,20.75
2015-07-02 10:34:34,10862,2015-07-02,10:34:34,2015-07-02 10:34:34,24702,southw_ckn_l,1,southw_ckn,L,20.75,20.75
2015-03-30 10:50:46,5247,2015-03-30,10:50:46,2015-03-30 10:50:46,11943,cali_ckn_m,1,cali_ckn,M,16.75,16.75
2015-06-17 10:52:26,9991,2015-06-17,10:52:26,2015-06-17 10:52:26,22718,cali_ckn_m,1,cali_ckn,M,16.75,16.75
2015-10-04 10:54:55,16439,2015-10-04,10:54:55,2015-10-04 10:54:55,37233,cali_ckn_m,1,cali_ckn,M,16.75,16.75
2015-02-25 10:54:03,3283,2015-02-25,10:54:03,2015-02-25 10:54:03,7448,four_cheese_l,2,four_cheese,L,17.95,35.90
2015-02-25 10:54:03,3283,2015-02-25,10:54:03,2015-02-25 10:54:03,7449,napolitana_s,1,napolitana,S,12.00,12.00


### Heat Map Validation results
- No December sales in 9:00 am hour, so Heat Map coloring is inaccurate
- Sales before 9:00 am in other months do seem to align with heat map coloring
- Conclusion: Heat Map coloring is using sum of all sales, and not the calculated field using the Chosen Month parameter
    
### Changed the calculation on the Heat Map color to use the correct calculated field, and successfully validated the results with the above numbers
    

## Validate units sold by pizza type

In [30]:
# merge december_line_item_df with pizza_types to bring in name and category for december
orders_category_df = pd.merge(left=december_line_item_df,right=pizza_types_df,on='pizza_type_id',how='left')
orders_category_df.head()

,order_id,date,time,order_date,order_details_id,pizza_id,quantity,pizza_type_id,size,price,line_cost,day_of_week,name,category,ingredients
0,19680,2015-12-01,12:24:32,2015-12-01 12:24:32,44786,hawaiian_m,1,hawaiian,M,13.25,13.25,Tuesday,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,19690,2015-12-01,13:24:16,2015-12-01 13:24:16,44803,hawaiian_m,1,hawaiian,M,13.25,13.25,Tuesday,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
2,19704,2015-12-01,16:22:17,2015-12-01 16:22:17,44838,hawaiian_m,1,hawaiian,M,13.25,13.25,Tuesday,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
3,19712,2015-12-01,17:45:53,2015-12-01 17:45:53,44853,hawaiian_m,1,hawaiian,M,13.25,13.25,Tuesday,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
4,19722,2015-12-01,19:21:05,2015-12-01 19:21:05,44875,hawaiian_m,1,hawaiian,M,13.25,13.25,Tuesday,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"


In [32]:
# group by category and name
category_name_df = orders_category_df.groupby(by=['category','name'])['quantity'].sum()
category_name_df

category  name                                      
Chicken   The Barbecue Chicken Pizza                    176
          The California Chicken Pizza                  207
          The Chicken Alfredo Pizza                      75
          The Chicken Pesto Pizza                        82
          The Southwest Chicken Pizza                   141
          The Thai Chicken Pizza                        204
Classic   The Big Meat Pizza                            157
          The Classic Deluxe Pizza                      197
          The Greek Pizza                               110
          The Hawaiian Pizza                            209
          The Italian Capocollo Pizza                   122
          The Napolitana Pizza                          103
          The Pepperoni Pizza                           183
          The Pepperoni, Mushroom, and Peppers Pizza    101
Supreme   The Brie Carre Pizza                           35
          The Calabrese Pizza                  

### Units sold by pizza type (name) and category checks with dashboard for Dec.